In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("G:/LiverCancer/predict_result/1_result.csv")

rater_a = df.iloc[:,6].values.tolist()
rater_b = [row.index(max(row)) for row in df.iloc[:, 1:6].values.tolist()]

temp, conf = quadratic_weighted_kappa(rater_a,rater_b)

In [ ]:
def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None): 
    
    rater_a = np.array(rater_a, dtype=int) 
    rater_b = np.array(rater_b, dtype=int) 
    
    assert(len(rater_a) == len(rater_b)) 
    
    if min_rating is None: min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None: max_rating = max(max(rater_a), max(rater_b))
        
    conf_mat = confusion_matrix(rater_a, rater_b, min_rating, max_rating)
    
    print(conf_mat)
    
    num_ratings = len(conf_mat) 
    num_scored_items = float(len(rater_a)) 
    
    hist_rater_a = histogram(rater_a, min_rating, max_rating) 
    hist_rater_b = histogram(rater_b, min_rating, max_rating) 
    
    numerator = 0.0 
    denominator = 0.0 
    
    for i in range(num_ratings): 
        
        for j in range(num_ratings): 
            
            expected_count = (hist_rater_a[i] * hist_rater_b[j] / num_scored_items) 
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0) 
            numerator += d * conf_mat[i][j] / num_scored_items 
            denominator += d * expected_count / num_scored_items 
            
    return 1.0 - numerator / denominator, conf_mat 

def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):

    num_ratings = int(max_rating - min_rating + 1)

    conf_mat = [[0 for i in range(num_ratings)] for j in range(num_ratings)]

    for a, b in zip(rater_a, rater_b):

        conf_mat[a - min_rating][b - min_rating] += 1

    return conf_mat

def histogram(ratings, min_rating=None, max_rating=None):

    num_ratings = int(max_rating - min_rating + 1)

    hist_ratings = [0 for x in range(num_ratings)]

    for r in ratings:

        hist_ratings[r - min_rating] += 1
    
    return hist_ratings